In [1]:
# dev of code to convert classic networks and adjust sigmas for a run of exp 00

In [2]:
from __future__ import print_function

In [3]:
import torch
import os 
import sys

sys.path.append("/home/felix/Research/Adversarial Research/FGN---Research/")
import Finite_Gaussian_Network_lib as fgnl
import Finite_Gaussian_Network_lib.fgn_helper_lib as fgnh

In [4]:
!gpustat

crescent  Mon Sep 30 16:18:56 2019
[0] GeForce GTX 1080 | 56'C,   0 % |  5906 /  8119 MB | richardso21(151M) enis(5743M)
[1] GeForce GTX 1080 | 66'C,  23 % |  6825 /  8119 MB | soumi(1343M) richardso21(483M) vietanh(593M) near(2269M) enis(113M) soumi(1125M) felix(887M)


In [5]:
# Define what device we are using
print("CUDA Available: ",torch.cuda.is_available())
use_cuda = False
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")
print("Using device:", device)

# manualy set cuda device
torch.cuda.set_device(0)

CUDA Available:  True
Using device: cpu


In [6]:
# load MNIST data
batch_size = 1250
(mnist_train_loader, mnist_val_loader, mnist_test_loader) = fgnh.mnist_dataloaders(batch_size)

lmbda_l2 = (4.0*0.1/len(mnist_train_loader.dataset))

In [7]:
run_dir = "/home/felix/Research/Adversarial Research/FGN---Research/Experiments/Exp-00_train_models-run2/"

In [8]:
# build a list of all the networks to convert
classic_models_list = []

for filename in os.listdir(run_dir):
    if ("classic" in filename) and ("_full.pth" in filename) and ("trained" in filename):
        classic_models_list.append(filename)

In [9]:
print(classic_models_list)

['2019-09-17 18:32:55.812514_[32, 32]_classic_1_trained_weights_full.pth', '2019-09-17 16:42:14.934314_[32, 32]_classic_0_trained_weights_full.pth', '2019-09-17 21:49:37.914325_[32, 32]_classic_2_trained_weights_full.pth']


In [10]:
# get the shared run params

with open(run_dir+"shared_parameters.txt", "r") as f:
    for line in f.readlines():
        l = line.split(" ")
        if l[0] == "drop_p":
            drop_p=float(l[1])
        
print("drop_p:", drop_p)


drop_p: 0.0


In [11]:
# get general fgn params
covar_type = "diag"
ordinal = float(2)
non_lin = False
noisy_centers = False
random_eval = False

# adjust sigmas params
adjust_sigmas_loss = fgnl.def_fgn_cross_ent_loss(lmbda_l2=0.0, lmbda_sigs=1e1*lmbda_l2)

In [12]:
# for each model in the run
for classic_model_path in classic_models_list:
    # load the classic model
    classic_model = torch.load(run_dir+classic_model_path, map_location=device)
    print(classic_model)

    # build the hidden layer that will be used to create the FGN
    hidden_layer_sizes = []
    for h in classic_model.hidden_layers.modules():
        if type(h) == torch.nn.modules.Linear:
            hidden_layer_sizes.append(h.out_features)
            
    fgn_model = fgnl.Feedforward_FGN_net(in_feats=28*28, out_feats=10, hidden_layer_sizes=hidden_layer_sizes, drop_p=drop_p,
                                     covar_type=covar_type, ordinal=ordinal, non_lin=non_lin, 
                                     noisy_centers=noisy_centers)     
    fgn_model.to(device)

    print(fgn_model)
    
    print("Converting model")
    # convert
    fgnl.convert_classic_to_fgn(classic_model=classic_model, fgn_model=fgn_model)
    print(fgn_model)
    
    
    print("Adjusting sigmas")
    # adjust sigmas
    adjust_sigmas_optimizer = torch.optim.RMSprop(zip(*filter(lambda p: p[1].requires_grad and "inv_covar" in p[0], 
                                                 fgn_model.named_parameters()))[1])
    
    converted_train_res = fgnh.train(fgn_model, mnist_train_loader, 
                             adjust_sigmas_loss, adjust_sigmas_optimizer, epochs=1, save_hist=2, verbose=True, 
                             pred_func=fgnh.cross_ent_pred_accuracy, test_loader=mnist_test_loader)
    
    print("Saving results")
    # name to save as
    converted_model_path = classic_model_path.replace("classic", "converted_adjusted_FGN")
    print(classic_model_path)
    print(run_dir+converted_model_path)
    
    fgn_model.eval() 
    
    # save model entirely (only if non_lin == False, otherwise pickle fails)
    if non_lin==False:
        torch.save(obj=fgn_model, f=run_dir+converted_model_path)

    # save model weights
    torch.save(fgn_model.state_dict(), run_dir+converted_model_path.replace("_full", "_state_dict"))

    # save training histories as pickle
    with open(run_dir+converted_model_path.replace("_full.pth", "_training_history.pckl"),"wb") as pickle_file:
            pickle.dump(converted_train_res,pickle_file)
    
    
    # (old) save as text file 
#     with open(run_dir+converted_model_path.replace("_full.pth", "_training_history.txt"), "w") as text_file:
#         for key in converted_train_res.keys():
#             text_file.write("{} {}\n".format(key, converted_train_res[key]))


Feedforward_Classic_net(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=784, out_features=32, bias=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (ib): BatchNorm1d(784, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fl): Linear(in_features=32, out_features=10, bias=True)
)
Feedforward_FGN_net(
  (hidden_layers): ModuleList(
    (0): FGN_layer()
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): FGN_layer()
    (3): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (ib): BatchNorm1d(784, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fl): FGN_layer()
)
Converting model
Feedforward_FGN_net(
  (hidden_layers): ModuleList(
    (0): FGN_layer()
    (1): Batch